# Mid-term evaluation report
**Team members**:
- Alex Scofield Teruel
- Hugo Taille Manikom
- François-Elie Ingwer

The following document details and justifies the architectural and implementation choices that have been made during the (first-half) of the development of the boolean circuits project.

## Development process

We have put into practice non-strict test-driven development. This is why some tests are skipped; in order not to clutter the output with failures of tests of functions that have still not been implemented, such as some functions from weeks four and five.

We use _Git_ as our VCS and have a private remote repository on _GitLab_. Given the limited-size of our team and that the project's size is still manageable, we have decided to develop directly on the _main_ branch, thus reducing the complexity of our project. This is subject to change as the project grows.

## Project Architecture

In order to maximise the modularity and extensability of the repository, whilst mantaining a cohesive structure well-adapted to the dimensions of the project, the code has been divided into three distinct parts:
- The modules _open_digraph.py_ and _adjacency_matrices.py_, in the _modules_ directory.
- Tests for the _open_digraph.py_ module (where all of the functions that are meant to be accessed when using this project are implemented) in the _open_digraph_tests.py_ file in the _tests_ directory
- Entry points, in the root of the repository, discussed below.

The repository contains three distinct entry points. 
1. _run_tests.py_: Conveniently runs all of the project's tests. For the moment these are all contained in a single file, but the choice to extract the entry point to this easily accessible test runner allows further tests to be ran automatically. One must be placed in the root directory of the repository in order to run this script.

2. _worksheet.py_: To be used as a scratchbook. Here is where the project can be easily used and manually tested, without cluttering the _modules_ directory. No new functionality should be implemented here.
3. _mid-term_eval.ipynb_: The current file. Demonstrates some of the fundamental functionalities of the project. Notably, adjacency matrices and composition of open directed graphs.

In the root of the repository there is also a _requirements.txt_ file containing all of the different python modules used. In order to minimise coupling these dependancies are kept at the strict minimum (currently 0!). There is also a _media_ directory containing the images used throughout this notebook.

Finally, each member of the team posseses their own _.gitignore_ file adapted to their specific development environment. In early commits this file was shared among the developers, but has been removed from the VCS for convenience.

### Usage guidelines
The _node_ class should not be accessed externally. In other words, the _API_ of this project currently consists of the public attributes and methods of the _open_digraph_ class. All public methods have been concisely documented directly within the source code. As the projeect grows a tool such as _Sphynx_ could be use to automatically generate docs, but for the time being we have considered that this would unnecessarily complexify the project.

**All functions, attributes and methods whose name begins with an undescore are private and thus should not be accessed directly!** Getter and setter methods are provided where relevant.

The tests (located in the _test_ directory) serve as simple examples of usage, showing how each function is expected to be called. However, it is important to note that it is preferable to use the _add_node()_ method of _open_digraph_ instead of adding the nodes manually, as is often done throughout the tests in order to make comparisons more explicit.

## Part 3 (Adjacency Matrices)

The _adjacency_matrices.py_ module provides a minimal interface to create random adjacency matrices corresponding to different types of graphs. It is used in _open_digraph.py_ to construct random instances of different types of graphs, subject to user-specified cardinality constraints. It is important to note that this module does not include any dependancies, and could thus be extended directly in other ways.

_adjacency_matrices.py_ only includes three functions: 
- _random_int_list(n, bound)_ 
- _random_dag_int_matrix(n, bound, null_diag = True)_
- _random_matrix(n, bound, null_diag=True, symmetric=False, oriented=False, dag=False)_

Given that this module is not meant to be accessed directly; since the public _API_ of the project is limited to the public attributes and methods of the _open_digraph_ class, the methods implemented in this module are not tested directly, but are instead tested through the methods of _open_digraph_ which use them.

For convenience, the implementation of the _adjacency_matrices.py_ module can be viewed directly by executing the cell below; or, of course, by opening the _adjacency_matrices.py_ file, located in the _modules_ directory.

In [1]:
import inspect
import modules.adjacency_matrices
module_source_code = inspect.getsource(modules.adjacency_matrices)
print(module_source_code)

import random

def random_int_list(n, bound):
    '''
    n: int; Number of elements in the list
    bound: int; Maximum possible integer
    Generates a random list of length n, with integers from 0 to bound
    '''
    return [random.randrange(0, bound+1) for _ in range(n)]

def random_matrix(n, bound,null_diag=True, symmetric=False, oriented=False):
    '''
    n: int; Number of elements in the list
    bound: int; Maximum possible integer
    null_diag: bool; If True, nodes cannot connect with themselves
    symmetric: bool; If True, generates a symmetric graph
    oriented: bool; If True, generates an oriented graph
    dag: bool; If True, generates a directides acyclic graph
    Generates a random matrix of size n, with integers from 0 to bound
    '''
    M = [random_int_list(n, bound) for _ in range(n)]

    if null_diag:
        for i in range(n):
            M[i][i] = 0

    if symmetric:
        for i in range(n):
            for j in range(i):
                M[j][i] = M[i]

## Part 6 (Composition)

Through the use of a class called _CompositionTest_ and two subclasses thereof, both parallel and sequential composition have been thoroughly tested for graphs such as the following:

### Graph 1
![gr1](media/gr1.png)

### Graph 2
![gr2](media/gr2.png)

### Parallel composition
![parallel](media/parallel_composition.png)

### Sequential composition
![sequential](media/sequential_composition.png)

For convenience the implementation of both types of compositions can be obtained by executing the cells below (as well as in _open_digraph.py_).

### iparallel

In [4]:
import inspect
import modules.open_digraph

def print_method_source(method):
    source_lines, _ = inspect.getsourcelines(method)
    for line in source_lines:
        print(line)

print_method_source(modules.open_digraph.open_digraph.iparallel)

    def iparallel(self, g):

        '''

        g: open_digraph; graph to be composed in parallel with self

        Modifies the current graph by composing it in parallel with the graph passed as a parameter

        '''

        g = g.copy()

        if self.get_nodes() != [] and g.get_nodes() != []:

            g.shift_indices(self.max_id() - g.min_id() + 1)



        self._inputs.extend(g.get_input_ids())

        self._outputs.extend(g.get_output_ids())



        for id, node in g.get_node_map().items():

            self._nodes[id] = node



### parallel (class method)

In [5]:
print_method_source(modules.open_digraph.open_digraph.parallel)

    @classmethod

    def parallel(cls, g1, g2):

        '''

        g1: open_digraph; first graph of composition

        g2: open_digraph; second graph of composition

        Returns the parallel composition of the parameter graphs

        Note that the operation is not strictly commutative, but will return isomorphic graphs

        '''

        composition = g1.copy()

        composition.iparallel(g2)

        return composition



### icompose

In [6]:
print_method_source(modules.open_digraph.open_digraph.icompose)

    def icompose(self, f):

        '''

        f: open_digraph; graph to be composed sequentially with self

        Modifies the current graph by composing it (if possible) in sequence with the graph passed as a parameter

        '''

        if len(self.get_input_ids()) != len(f.get_output_ids()):

            raise ValueError("Number of inputs of the first graph does not match the number of outputs of the second graph.")

        

        f = f.copy()



        if self.get_nodes() != [] and f.get_nodes() != []:

            self.shift_indices(f.max_id() - self.min_id() + 1)

        

        for id, node in f.get_node_map().items():

            self._nodes[id] = node.copy()





        prev_inputs = self.get_input_ids()

        self._inputs = f.get_input_ids()

        for input_id, output_id in zip(prev_inputs, f.get_output_ids()):

            self.add_edge(output_id, input_id)



### compose (class method)

In [7]:
print_method_source(modules.open_digraph.open_digraph.compose)

    @classmethod

    def compose(cls, g1, g2):

        '''

        g1: open_digraph; first graph of composition

        g2: open_digraph; second graph of composition

        Returns the composition of the parameter graphs

        Note that the operation is not strictly commutative

        '''

        g1 = g1.copy()

        g1.icompose(g2)

        return g1



The code concerning connected components can be obtained by executing the following cell (or in _open_digraph.py_). In order to find the connected components of a graph, we use a DFS.

In [8]:
print_method_source(modules.open_digraph.open_digraph.separate_connected_components)

    def separate_connected_components(self):

        '''

        Separates the circuit into its connected components.

        Returns a list of open_digraph objects representing the connected components.

        '''

        connected_components = []

        visited = set()



        def dfs(node_id, component):

            if node_id not in visited:

                visited.add(node_id)

                component.append(node_id)

                for child_id in self._nodes[node_id].get_children():

                    dfs(child_id, component)

                for parent_id in self._nodes[node_id].get_parents():

                    dfs(parent_id, component)



        for node_id in self.get_node_ids():

            if node_id not in visited:

                component = []

                dfs(node_id, component)

                connected_components.append(component)



        components_graphs = []

        for component in connected_components:

            component_graph